In [1]:
import pandas as pd

In [ ]:
#task 1
attendance_raw = {
    "S001": {"cohort": "alpha", "attended_sessions": 5, "expected_sessions": 6},
    "S002": {"cohort": "alpha", "attended_sessions": 6, "expected_sessions": 6},
    "S003": {"cohort": "beta", "attended_sessions": 4, "expected_sessions": 6},
    "S004": {"cohort": "beta", "attended_sessions": 5, "expected_sessions": 6},
    "S005": {"cohort": "gamma", "attended_sessions": 2, "expected_sessions": 6},
    "S006": {"cohort": "gamma", "attended_sessions": 3, "expected_sessions": 6},
    "S007": {"cohort": "alpha", "attended_sessions": 6, "expected_sessions": 6},
    "S008": {"cohort": "beta", "attended_sessions": 1, "expected_sessions": 6},
    "S009": {"cohort": "gamma", "attended_sessions": 4, "expected_sessions": 6},
    "S010": {"cohort": "alpha", "attended_sessions": 3, "expected_sessions": 6},
    "S011": {"cohort": "beta", "attended_sessions": 1, "expected_sessions": 6},
    "S012": {"cohort": "gamma", "attended_sessions": 5, "expected_sessions": 6},
    "S013": {"cohort": "alpha", "attended_sessions": 4, "expected_sessions": 6},
    "S014": {"cohort": "beta", "attended_sessions": 2, "expected_sessions": 6},
    "S015": {"cohort": "gamma", "attended_sessions": 3, "expected_sessions": 6},    
    "S016": {"cohort": "alpha", "attended_sessions": 5, "expected_sessions": 6},
    "S017": {"cohort": "beta", "attended_sessions": 4, "expected_sessions": 6},
    "S018": {"cohort": "gamma", "attended_sessions": 2, "expected_sessions": 6},
    "S019": {"cohort": "alpha", "attended_sessions": 6, "expected_sessions": 6},
    "S020": {"cohort": "beta", "attended_sessions": 3, "expected_sessions": 6},
    "S021": {"cohort": "gamma", "attended_sessions": 1, "expected_sessions": 6},
    "S022": {"cohort": "alpha", "attended_sessions": 4, "expected_sessions": 6},
    "S023": {"cohort": "beta", "attended_sessions": 5, "expected_sessions": 6},
    "S024": {"cohort": "gamma", "attended_sessions": 6, "expected_sessions": 6},
    }
attendance = pd.DataFrame(attendance_raw).T
print(attendance.head())
print(attendance.info())


     cohort attended_sessions expected_sessions
S001  alpha                 5                 6
S002  alpha                 6                 6
S003   beta                 4                 6
S004   beta                 5                 6
S005  gamma                 2                 6
<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, S001 to S024
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   cohort             24 non-null     object
 1   attended_sessions  24 non-null     object
 2   expected_sessions  24 non-null     object
dtypes: object(3)
memory usage: 768.0+ bytes
None


In [ ]:
#task 2
attendance_indexed = attendance.copy() 
excused_absences = pd.Series([1, 0, 0, 1 , 1, 2, 0, 1, 2, 3], index=["S001", "S002", "S103", "S004", "S005", "S006", "S031", "S012", "S013", "S025"])
attendance_indexed["adjusted_attendance"] = attendance_indexed["attended_sessions"] + excused_absences
print(attendance_indexed["adjusted_attendance"])
attendance_indexed["adjusted_attendance"] = attendance_indexed["adjusted_attendance"].fillna(attendance_indexed["attended_sessions"])
print(attendance_indexed["adjusted_attendance"])


S001    6.0
S002    6.0
S003    NaN
S004    6.0
S005    3.0
S006    5.0
S007    NaN
S008    NaN
S009    NaN
S010    NaN
S011    NaN
S012    6.0
S013    6.0
S014    NaN
S015    NaN
S016    NaN
S017    NaN
S018    NaN
S019    NaN
S020    NaN
S021    NaN
S022    NaN
S023    NaN
S024    NaN
Name: adjusted_attendance, dtype: object
S001    6.0
S002    6.0
S003    4.0
S004    6.0
S005    3.0
S006    5.0
S007    6.0
S008    1.0
S009    4.0
S010    3.0
S011    1.0
S012    6.0
S013    6.0
S014    2.0
S015    3.0
S016    5.0
S017    4.0
S018    2.0
S019    6.0
S020    3.0
S021    1.0
S022    4.0
S023    5.0
S024    6.0
Name: adjusted_attendance, dtype: float64
     cohort attended_sessions expected_sessions  adjusted_attendance
S001  alpha                 5                 6                  6.0
S002  alpha                 6                 6                  6.0
S003   beta                 4                 6                  4.0
S004   beta                 5                 6                  

C:\Users\user\AppData\Local\Temp\ipykernel_8024\552475984.py:7: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  attendance_indexed["adjusted_attendance"] = attendance_indexed["adjusted_attendance"].fillna(attendance_indexed["attended_sessions"])


In [ ]:
print(attendance_indexed)

     cohort attended_sessions expected_sessions  adjusted_attendance
S001  alpha                 5                 6                  6.0
S002  alpha                 6                 6                  6.0
S003   beta                 4                 6                  4.0
S004   beta                 5                 6                  6.0
S005  gamma                 2                 6                  3.0
S006  gamma                 3                 6                  5.0
S007  alpha                 6                 6                  6.0
S008   beta                 1                 6                  1.0
S009  gamma                 4                 6                  4.0
S010  alpha                 3                 6                  3.0
S011   beta                 1                 6                  1.0
S012  gamma                 5                 6                  6.0
S013  alpha                 4                 6                  6.0
S014   beta                 2     

In [23]:
#task 3
attendance_mdf = attendance_indexed.copy()

attendance_mdf.to_csv("attendance_modified.csv")

print(attendance_mdf)

     cohort attended_sessions expected_sessions  adjusted_attendance
S001  alpha                 5                 6                  6.0
S002  alpha                 6                 6                  6.0
S003   beta                 4                 6                  4.0
S004   beta                 5                 6                  6.0
S005  gamma                 2                 6                  3.0
S006  gamma                 3                 6                  5.0
S007  alpha                 6                 6                  6.0
S008   beta                 1                 6                  1.0
S009  gamma                 4                 6                  4.0
S010  alpha                 3                 6                  3.0
S011   beta                 1                 6                  1.0
S012  gamma                 5                 6                  6.0
S013  alpha                 4                 6                  6.0
S014   beta                 2     

In [24]:
attendance_mdf = pd.read_csv("attendance_modified.csv", index_col=0)
print(attendance_mdf)

        cohort  attended_sessions  expected_sessions  adjusted_attendance
S001    alpha                   5                  6                  6.0
S002     alpha                  6                  6                  6.0
S003      beta                  4                  6                  4.0
S004      BETA                  5                  6                  6.0
S005    gamma                   2                  6                  3.0
S006     gamma                  3                  6                  5.0
S007     Alpha                  6                  6                  6.0
S008      beta                  1                  6                  1.0
S009     gamma                  4                  6                  4.0
S010     alpha                  3                  6                  3.0
S011      beta                  1                  6                  1.0
S012   gamma                    5                  6                  6.0
S013    alpha                   4     

In [ ]:
attendance_mdf_cleaned = attendance_mdf.copy()
attendance_mdf_cleaned["cohort"] = attendance_mdf["cohort"].str.strip().str.lower()
print(attendance_mdf_cleaned["cohort"])
#check

S001    alpha
S002    alpha
S003     beta
S004     beta
S005    gamma
S006    gamma
S007    alpha
S008     beta
S009    gamma
S010    alpha
S011     beta
S012    gamma
S013    alpha
S014     beta
S015    gamma
S016    alpha
S017     beta
S018    gamma
S019    alpha
S020     beta
S021    gamma
S022    alpha
S023     beta
S024    gamma
Name: cohort, dtype: object


In [36]:
#task 4

low_attendance = attendance_mdf_cleaned[attendance_mdf_cleaned["attended_sessions"] < attendance_mdf_cleaned["expected_sessions"]]
print(low_attendance)

low_attendance.groupby("cohort")["attended_sessions"].mean()

     cohort  attended_sessions  expected_sessions  adjusted_attendance
S001  alpha                  5                  6                  6.0
S003   beta                  4                  6                  4.0
S004   beta                  5                  6                  6.0
S005  gamma                  2                  6                  3.0
S006  gamma                  3                  6                  5.0
S008   beta                  1                  6                  1.0
S009  gamma                  4                  6                  4.0
S010  alpha                  3                  6                  3.0
S011   beta                  1                  6                  1.0
S012  gamma                  5                  6                  6.0
S013  alpha                  4                  6                  6.0
S014   beta                  2                  6                  2.0
S015  gamma                  3                  6                  3.0
S016  

cohort
alpha    4.200000
beta     3.125000
gamma    2.857143
Name: attended_sessions, dtype: float64

In [37]:
#task 5

attendance_mdf_cleaned["attendance_ok"] = attendance_mdf_cleaned["attended_sessions"] >= attendance_mdf_cleaned["expected_sessions"]
print(attendance_mdf_cleaned[["attended_sessions", "expected_sessions", "attendance_ok"]])

      attended_sessions  expected_sessions  attendance_ok
S001                  5                  6          False
S002                  6                  6           True
S003                  4                  6          False
S004                  5                  6          False
S005                  2                  6          False
S006                  3                  6          False
S007                  6                  6           True
S008                  1                  6          False
S009                  4                  6          False
S010                  3                  6          False
S011                  1                  6          False
S012                  5                  6          False
S013                  4                  6          False
S014                  2                  6          False
S015                  3                  6          False
S016                  5                  6          False
S017          